# Suspicious PowerShell Invocations - Specific
Detects suspicious PowerShell invocation command parameters

## Rule Content
```
- title: Suspicious PowerShell Invocations - Specific
  id: fce5f582-cc00-41e1-941a-c6fabf0fdb8c
  status: experimental
  description: Detects suspicious PowerShell invocation command parameters
  tags:
  - attack.execution
  - attack.t1086
  author: Florian Roth (rule)
  logsource:
    product: windows
    service: powershell
    category: null
  detection:
    keywords:
      Message:
      - '* -nop -w hidden -c * [Convert]::FromBase64String*'
      - '* -w hidden -noni -nop -c "iex(New-Object*'
      - '* -w hidden -ep bypass -Enc*'
      - '*powershell.exe reg add HKCU\software\microsoft\windows\currentversion\run*'
      - '*bypass -noprofile -windowstyle hidden (new-object system.net.webclient).download*'
      - '*iex(New-Object Net.WebClient).Download*'
    condition: keywords
  falsepositives:
  - Penetration tests
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-powershell-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='Message.keyword:(*\ \-nop\ \-w\ hidden\ \-c\ *\ \[Convert\]\:\:FromBase64String* OR *\ \-w\ hidden\ \-noni\ \-nop\ \-c\ \"iex\(New\-Object* OR *\ \-w\ hidden\ \-ep\ bypass\ \-Enc* OR *powershell.exe\ reg\ add\ HKCU\\software\\microsoft\\windows\\currentversion\\run* OR *bypass\ \-noprofile\ \-windowstyle\ hidden\ \(new\-object\ system.net.webclient\).download* OR *iex\(New\-Object\ Net.WebClient\).Download*)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()